In [1]:
import sys
import time
import json
from alpyne.client.alpyne_client import AlpyneClient
from stable_baselines3 import A2C
from stable_baselines3.common.utils import set_random_seed
from stable_baselines3.common.env_util import make_vec_env

sys.path.append("../..")
from thesis.envs.matrix_routing_centralized import MatrixRoutingCentral
from thesis.policies.routing_attention import RoutingFE
from thesis.policies.ppo_ac_attention import AttentionACPolicy

seed = 42
set_random_seed(seed)

In [2]:
models_dir = "../../models/MiniMatrix_Routing_MA"
logdir = "../../logs/MiniMatrix_Routing_MA"
fleetsize = 6
max_fleetsize = 10
run_name = f"A2C-{fleetsize}-{max_fleetsize}-{time.strftime('%d_%m-%H_%M_%S')}-{seed}"

In [3]:
env_args = dict(
        reward_target = 1, 
        reward_distance = 0.05,
        reward_block = -0.5, 
        dispatchinginterval=30,
        routinginterval = 2,
        withCollisions = False,
        blockTimeout = 5,
        routingOnNode = False,
        # coordinates = False,
        includeNodesInReach = True,
    )

ppo_args = dict(
)
fe_args = dict(
    max_fleetsize=max_fleetsize,
    embed_dim = 64,
    n_heads = 8,
    depth = 8
)
net_arch = [dict(pi = [], vf = [])]

hparams = dict(
    fleetsize = fleetsize,
    max_fleetsize = max_fleetsize,
    env_args = env_args,
    ppo_args = ppo_args,
    fe_args = fe_args,
    net_arch = net_arch
)
with open(f"{models_dir}/{run_name}.json", 'w') as outfile:
    json.dump(hparams, outfile, indent = 3)

In [4]:
i = [0]

client = AlpyneClient("../../envs/MiniMatrix.zip", port=51140)

env = make_vec_env(MatrixRoutingCentral, 8, env_kwargs=dict(
    max_seconds = 60*60, 
    fleetsize = fleetsize, 
    max_fleetsize=max_fleetsize, 
    config_args = env_args,
    counter = i,
    client = client
))

c:\Users\Wegma\.conda\envs\thesis\lib\site-packages\alpyne\client\utils.py:124: UserWarning: Unzipping to temporary directory (C:\Users\Wegma\AppData\Local\Temp\alpyne_82296_7dc4g9f4)
  warn(f"Unzipping to temporary directory ({tmp_dir})")


In [5]:
model =A2C(
    AttentionACPolicy,
    env, 
    tensorboard_log= logdir,
    device = "cuda",
    policy_kwargs=dict(
        net_arch = net_arch,
        features_extractor_class=RoutingFE, 
        features_extractor_kwargs=fe_args
        ),
    **ppo_args,
    )

In [6]:
model = A2C.load(f"{models_dir}/A2C-6-10-10_08-17_31_07-42-150000.zip", env)

In [7]:
TIMESTEPS = 50000
for i in range(1, 15):
    model.learn(total_timesteps=TIMESTEPS, reset_num_timesteps=False, tb_log_name=run_name)#,callback=MiniMatrixCallback())
    model.save(f"{models_dir}/{run_name}-{TIMESTEPS * i}")

ModelError: ModelError[path=http://127.0.0.1:51140/runs/6/progress, status=500, error=Request failed., msg=('<html><head><title>Grizzly 3.0.0</title><style><!--div.header '
 '{font-family:Tahoma,Arial,sans-serif;color:white;background-color:#003300;font-size:22px;-moz-border-radius-topleft: '
 '10px;border-top-left-radius: 10px;-moz-border-radius-topright: '
 '10px;border-top-right-radius: 10px;padding-left: 5px}div.body '
 '{font-family:Tahoma,Arial,sans-serif;color:black;background-color:#FFFFCC;font-size:16px;padding-top:10px;padding-bottom:10px;padding-left:10px}div.footer '
 '{font-family:Tahoma,Arial,sans-serif;color:white;background-color:#666633;font-size:14px;-moz-border-radius-bottomleft: '
 '10px;border-bottom-left-radius: 10px;-moz-border-radius-bottomright: '
 '10px;border-bottom-right-radius: 10px;padding-left: 5px}BODY '
 '{font-family:Tahoma,Arial,sans-serif;color:black;background-color:white;}B '
 '{font-family:Tahoma,Arial,sans-serif;color:black;}A {color : black;}HR 

In [ ]:
env.envs[1].env.run.get_state()

(<RunStatus.FAILED: 'FAILED'>,
 {'reason': 'MODEL_ERROR',
  'next_event_time': 78.5,
  'episode_count': 3,
  'step_count': 44,
  'model_time': 78.5})

In [ ]:
env.envs[7].env.run.get_observation()

[obs:DOUBLE_ARRAY=[[1.0, 1.0, 0.42045454545454547, 0.8837209302325582, 0.42045454545454547, 0.8837209302325582, 0.6022727272727273, 1.0, 0.32954545454545453, 0.9395348837209302, 0.0, 0.0, 0.4772727272727273, 0.8837209302325582, 0.42045454545454547, 0.5581395348837209], [1.0, 1.0, 0.42045454545454547, 0.4418604651162791, 0.42045454545454547, 0.4418604651162791, 0.7613636363636364, 1.0, 0.32954545454545453, 0.49767441860465117, 0.0, 0.0, 0.4772727272727273, 0.4418604651162791, 0.0, 0.0], [1.0, 0.0, 0.7613636363636364, 0.8837209302325582, 0.7909090909090909, 0.7209302325581395, 0.7909090909090909, 0.7209302325581395, 0.0, 0.0, 0.8181818181818182, 0.8837209302325582, 0.0, 0.0, 0.0, 0.0], [1.0, 1.0, 0.4772727272727273, 1.0, 0.4772727272727273, 0.8837209302325582, 1.0, 0.8837209302325582, 0.42045454545454547, 0.8837209302325582, 0.0, 0.0, 0.5454545454545454, 0.8837209302325582, 0.4772727272727273, 0.5581395348837209], [1.0, 1.0, 0.9431818181818182, 1.0, 0.9431818181818182, 0.8837209302325582

In [ ]:
for i in range(15, 30):
    model.learn(total_timesteps=TIMESTEPS, reset_num_timesteps=False, tb_log_name=run_name)#,callback=MiniMatrixCallback())
    model.save(f"{models_dir}/{run_name}-{TIMESTEPS * i}")